Initializing Table

In [17]:
import tabula
import pandas as pd

#top,left,bottom,right of box
box =  [1.97, .11, 2.79, 7.77]
options = {'names': ["Last Raced", "Pgm", "Horse Name (Jockey)", "Wgt", "M/E", "PP", "Start", "1/4", "3/8", "Str", "Fin", "Odds", "Comments"]}
for i in range(0,len(box)):
    box[i] *= 72
df = tabula.read_pdf("equibaseFile.pdf", pages = 1, area = [box])

In [18]:
data = df[0]
#data = data.drop(index = 0)
data

,Last Raced,Unnamed: 0,Pgm,Horse Name (Jockey),Wgt M/E,PP,Start,1/4,3/8,Str,Fin,Odds,Comments
0,2 1,NaN,NaN,NaN,NaN,NaN,NaN,Head,Head,2,Nose,NaN,NaN
1,2Jul22 MTH,NaN,4.0,"Wild Mule (Jimenez, Albin)",118 L,4.0,3.0,2,2,2,1,0.90*,"battled back, got nod"
2,6 1,NaN,NaN,NaN,NaN,NaN,NaN,3,4,Head,3/4,NaN,NaN
3,2Jul22 MTH,NaN,6.0,"Rol Again Dancer (Rodriguez, Angel)",118 L,5.0,2.0,3,3,1,2,1.50,"vied 3w, led, missed"
4,2 1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Head,3/4,NaN,NaN
5,30Jun22 DEL,NaN,1.0,"Bright Forecast (Ferrer, Jose)",118 L,1.0,5.0,5,5,4,3,4.70,"off slowly, rallied"
6,1 1,NaN,NaN,NaN,NaN,NaN,NaN,4,2 1/2,NaN,2,NaN,NaN
7,22Jul22 MTH,NaN,2.0,"Running Right by U (Vargas, Jr., Jorge)",118 L,2.0,4.0,4,4,5,4,14.40,"inside, no rally"
8,5 7,NaN,NaN,NaN,NaN,NaN,NaN,1/2,1/2,3,NaN,NaN,NaN
9,30Jul22 MTH,NaN,3.0,"Run Happy Pappy (Torres, Jomar)",118 L b,3.0,1.0,1,1,3,5,14.10,"dueled inside, faded"


Splitting Wgt & M/E

In [3]:
data[['Wgt', 'M/E']] = data['Wgt M/E'].str.split(' ', 1,expand = True)
data

,Last Raced,Unnamed: 0,Pgm,Horse Name (Jockey),Wgt M/E,PP,Start,1/4,3/8,Str,Fin,Odds,Comments,Wgt,M/E
0,2 1,NaN,NaN,NaN,NaN,NaN,NaN,Head,Head,2,Nose,NaN,NaN,NaN,NaN
1,2Jul22 MTH,NaN,4.0,"Wild Mule (Jimenez, Albin)",118 L,4.0,3.0,2,2,2,1,0.90*,"battled back, got nod",118,L
2,6 1,NaN,NaN,NaN,NaN,NaN,NaN,3,4,Head,3/4,NaN,NaN,NaN,NaN
3,2Jul22 MTH,NaN,6.0,"Rol Again Dancer (Rodriguez, Angel)",118 L,5.0,2.0,3,3,1,2,1.50,"vied 3w, led, missed",118,L
4,2 1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Head,3/4,NaN,NaN,NaN,NaN
5,30Jun22 DEL,NaN,1.0,"Bright Forecast (Ferrer, Jose)",118 L,1.0,5.0,5,5,4,3,4.70,"off slowly, rallied",118,L
6,1 1,NaN,NaN,NaN,NaN,NaN,NaN,4,2 1/2,NaN,2,NaN,NaN,NaN,NaN
7,22Jul22 MTH,NaN,2.0,"Running Right by U (Vargas, Jr., Jorge)",118 L,2.0,4.0,4,4,5,4,14.40,"inside, no rally",118,L
8,5 7,NaN,NaN,NaN,NaN,NaN,NaN,1/2,1/2,3,NaN,NaN,NaN,NaN,NaN
9,30Jul22 MTH,NaN,3.0,"Run Happy Pappy (Torres, Jomar)",118 L b,3.0,1.0,1,1,3,5,14.10,"dueled inside, faded",118,L b


Getting Table Areas

In [64]:
#Function Definitions
from pathlib import Path
from typing import Iterable, Any

from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer


def extract_to_df(pages: Any):
    cols = ["Page","Text", "x_1", "y_1", "x_2", "y_2"]
    words = ["Last Raced", "Fractional Times", "Pgm Horse Name", "Fin", "Trainers"]
    text_loc_df = pd.DataFrame(columns = cols)
    
    page_counter = 0
    for page_layout in pages:
        page_counter += 1
        word_counter = 0
        for element in page_layout:
            if isinstance(element, LTTextContainer):
                element_text = element.get_text()
                for word in words:
                    if(element_text.__contains__(word)):
                        loc_nums = []
                        for num in element.bbox:
                            loc_nums.append(num) #Adding binding box numbers
                        text_loc_df.loc[len(text_loc_df.index)] = [page_counter,
                                                                     word,
                                                                     loc_nums[0], 
                                                                     loc_nums[1], 
                                                                     loc_nums[2], 
                                                                     loc_nums[3]]
    return text_loc_df

In [65]:
#Extracting pages for table locations

pages = extract_pages("equibaseFile.pdf")
text_loc_df = extract_to_df(pages)
text_loc_df

The PDF <_io.BufferedReader name='equibaseFile.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case


,Page,Text,x_1,y_1,x_2,y_2
0,1,Last Raced,9.920,593.754,57.893,650.314
1,1,Pgm Horse Name,73.514,642.314,176.908,650.314
2,1,Fin,438.062,593.754,455.958,650.314
3,1,Fractional Times,9.920,558.454,154.659,584.454
4,1,Fin,231.019,576.454,301.226,584.454
...,...,...,...,...,...,...
83,11,Fin,438.062,593.754,458.634,650.314
84,11,Fin,231.019,576.454,301.226,584.454
85,11,Pgm Horse Name,158.437,332.229,226.679,340.229
86,11,Fin,389.760,283.669,410.332,340.229


In [66]:
#Data Cleaning

#Selecting only page 1

text_loc_df = text_loc_df.loc[text_loc_df['Page'] == 1] 


#Saving only last instance of fin; Lowest y_1

fin_df = text_loc_df.loc[text_loc_df['Text'] == "Fin"] #df with only Fin
fin_df = fin_df.sort_values(by=['y_1']) #Sort by y_1 descending
fin_row = fin_df.iloc[0] #Save 1st row
text_loc_df = text_loc_df.loc[text_loc_df['Text'] != "Fin"] #Delete Fin rows
text_loc_df = text_loc_df.append(fin_row) #Append right Fin row


#Saving only last instance of Pgm Horse Name; Lowest y_1

pgm_df = text_loc_df.loc[text_loc_df['Text'] == "Pgm Horse Name"] #df with only Fin
pgm_df = pgm_df.sort_values(by=['y_1']) #Sort by y_1 descending
pgm_row = pgm_df.iloc[0] #Save 1st row
text_loc_df = text_loc_df.loc[text_loc_df['Text'] != "Pgm Horse Name"] #Delete Fin rows
text_loc_df = text_loc_df.append(pgm_row) #Append right Fin row

#Resort table by index
text_loc_df = text_loc_df.sort_index()
text_loc_df

,Page,Text,x_1,y_1,x_2,y_2
0,1,Last Raced,9.920,593.754,57.893,650.314
3,1,Fractional Times,9.920,558.454,154.659,584.454
5,1,Pgm Horse Name,158.437,372.729,226.679,380.729
6,1,Fin,389.760,324.169,407.656,380.729
7,1,Trainers,9.920,303.535,390.629,311.535


In [72]:
#Creating table location dataframe
df = text_loc_df
cols = ["Page","Table",'Top','Left','Bottom','Right']
table_loc_df = pd.DataFrame(columns = cols)

#For each page
page = 1
#Top Table
y_2 = df.loc[df['Text'] == "Last Raced",'y_2'].tolist()[0] #Getting y_2 value
top = (((729 - y_2) * 11)/792) *72

left = 7.92
right = 559.44

y_1 = df.loc[df['Text'] == "Fractional Times",'y_1'].tolist()[0]
bottom = (((792 - y_1) * 11)/792 + .45) *72

#Adding Table 1 to df
table_loc_df.loc[len(table_loc_df.index)] =[page,1,top,left,bottom,right]

#Bottom Table
y_2 = df.loc[df['Text'] == "Pgm Horse Name",'y_2'].tolist()[0]
top = (((729 - y_2) * 11)/792) * 72

left = 126
right = 163

y_1 = df.loc[df['Text'] == "Trainers",'y_1'].tolist()[0]
bottom = (((792 - y_1) * 11)/792 + .45 ) * 72


#Adding Table 2 to df
table_loc_df.loc[len(table_loc_df.index)] =[page,2,top,left,bottom,right]


In [73]:
table_loc_df

,Page,Table,Top,Left,Bottom,Right
0,1.0,1.0,78.686,7.92,265.946,559.44
1,1.0,2.0,348.271,126.00,520.865,163.00
